# Visualizing the model

In this document we will visualize how the agent picks its routes throughout a day

In [71]:
import folium as folium
import eindhovenDists3 as dists
import osmnx as ox
from IPython.display import display


### A 'dumb' agent

In simulating a working-age adult, we have a preset residence and office for them

After working, this person will want to go to a random amount of locations based on likelyhood for a given type. In this example, some kind of supermarket or convenience store.

The 'dumb' approach is to pick the nearest store from the office.

In [72]:
data, all_pairs_distances = dists.initialize()

graph = data['graph']

# Pick a random residence from the data
residence = data['residences'].sample(1).iloc[0]

# pick a random office from the data
office = data['offices'].sample(1).iloc[0]

# Create a route between the residence and office
route1 = ox.shortest_path(graph, residence['nearest_node'], office['nearest_node'], weight='length')

# Get the nearest shops to the office
shops = dists.get_nearest_amenities_from_node(
    graph,
    office['nearest_node'],
    ('supermarket', 'convenience'),
    max_count=5)

# Create a route from the office to the nearest shop
if not shops.empty:
    shop = shops.iloc[0]
    route2 = ox.shortest_path(graph, office['nearest_node'], shop['nearest_node'], weight='length')

# Create a route from the shop back to the residence
route3 = ox.shortest_path(graph, shop['nearest_node'], residence['nearest_node'], weight='length')

# Create a folium map centered around the residence
m = folium.Map(location=[residence['centroid'].y, residence['centroid'].x], zoom_start=14)
# Add the routes to the map
folium.PolyLine(
    locations=[(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in route1],
    color='blue',
    weight=5,
    opacity=0.7
).add_to(m)
if not shops.empty:
    folium.PolyLine(
        locations=[(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in route2],
        color='green',
        weight=5,
        opacity=0.7
    ).add_to(m)
folium.PolyLine(
    locations=[(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in route3],
    color='red',
    weight=5,
    opacity=0.7
).add_to(m)
# Add markers for the residence, office, and shop
folium.Marker(
    location=[residence['centroid'].y, residence['centroid'].x],
    popup='Residence',
    icon=folium.Icon(color='blue', icon='home')
).add_to(m)
folium.Marker(
    location=[office['centroid'].y, office['centroid'].x],
    popup='Office',
    icon=folium.Icon(color='green', icon='briefcase')
).add_to(m)

# Add markers for the shops if any
if not shops.empty:
    for _, shop in shops.iterrows():
        folium.Marker(
            location=[shop['centroid'].y, shop['centroid'].x],
            popup=shop['name'],
            icon=folium.Icon(color='red', icon='shopping-cart')
        ).add_to(m)

# Display the map
display(m)
# Save the map to an HTML file
m.save('model_visualization1.html')

Initializing data...
Data already loaded.


### The smarter approach

In this approach, the agent will try to save some time by not just visiting the nearest place from their current location.

Technically, the most optimized travel time in a day would be achieved by computing the result to the so-called Travelling Salesman Problem. However, this is an NP-hard problem that is both difficult to solve on a graph with as many nodes as Eindhoven by a machine and an even more unrealistic calculation to expect a normal person to do just to optimize their daily routine.

We will aim for a simpler and more realistic approach: Whenever a person is about to visit their last place of the day before heading home, we will look at all the places along the route and pick the one that requires the smallest detour. Let's see how this changes the earlier agent's route.

In [73]:
shops_inbetween = dists.get_nearest_amenities_inbetween(
    graph,
    office['nearest_node'],
    residence['nearest_node'],
    ('supermarket', 'convenience'),
    max_count=5
    )
# Create a route from the office to the residence, but with the nearest shop in between
if not shops_inbetween.empty:
    shop = shops_inbetween.iloc[0]
    route2 = ox.shortest_path(graph, office['nearest_node'], shop['nearest_node'], weight='length')
    route3 = ox.shortest_path(graph, shop['nearest_node'], residence['nearest_node'], weight='length')

    # Create a new folium map centered around the residence
    m2 = folium.Map(location=[residence['centroid'].y, residence['centroid'].x], zoom_start=14)
    
    # Add the routes to the map
    folium.PolyLine(
        locations=[(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in route1],
        color='blue',
        weight=5,
        opacity=0.7
    ).add_to(m2)
    
    folium.PolyLine(
        locations=[(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in route2],
        color='green',
        weight=5,
        opacity=0.7
    ).add_to(m2)
    
    folium.PolyLine(
        locations=[(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in route3],
        color='red',
        weight=5,
        opacity=0.7
    ).add_to(m2)
    
    # Add markers for the residence, office, and shop
    folium.Marker(
        location=[residence['centroid'].y, residence['centroid'].x],
        popup='Residence',
        icon=folium.Icon(color='blue', icon='home')
    ).add_to(m2)
    
    folium.Marker(
        location=[office['centroid'].y, office['centroid'].x],
        popup='Office',
        icon=folium.Icon(color='green', icon='briefcase')
    ).add_to(m2)

    # Add markers for the shops if any
    if not shops_inbetween.empty:
        for _, shop in shops_inbetween.iterrows():
            folium.Marker(
                location=[shop['centroid'].y, shop['centroid'].x],
                popup=shop['name'],
                icon=folium.Icon(color='red', icon='shopping-cart')
            ).add_to(m2)

    # Display the map
    display(m2)
    
    # Save the map to an HTML file
    m2.save('model_visualization2.html')

### Note

This notebook can be rerun to get different random results